In [20]:
import config
#DB instance creation
import mysql.connector
from mysql.connector import errorcode
import requests

# # connecting to the database using 'connect()' method
# # it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host=config.host,
    user=config.user,
    passwd=config.password
)

# # create cursor
cursor = cnx.cursor()

# ## create data base function ##


def create_database(cursor, database):
    try:
        # fucntion to create a db name
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    # return error message if failing to open a database
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)


# try to create the DB - fail if exists
try:
    cursor.execute("USE {}".format(config.DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(config.DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, config.DB_NAME)
        print("Database {} created successfully.".format(config.DB_NAME))
        cnx.database = config.DB_NAME
    else:
        print(err)
        exit(1)
      
#method for connecting to DB, returns the connection and the cursor
def connect_to_db():
    cnx = mysql.connector.connect(
    host=config.host, user=config.user, passwd=config.password, database=config.DB_NAME)
    cursor = cnx.cursor()
    return cnx, cursor



In [23]:
def create_tables():
    # creating the queries for the actual tables setting business id as a primary key and a foreign key
    DB_NAME = 'movie_project'
    TABLES = {}
    TABLES['movies'] = (
        "CREATE TABLE movies ("
        "  movie_id varchar(24) NOT NULL,"
        "  title varchar(12) NOT NULL,"
        "  revenue float(24) ,"
        "  release_date datetime,"
        "  vote_count int(12),"
        "  vote_avg float(12),"
        "  tag_line varchar(256),"
        "  PRIMARY KEY (movie_id)"
        ") ENGINE=InnoDB")

    




    TABLES['genres'] = """
                        CREATE TABLE genres(
                            genre_id varchar(24) NOT NULL,
                            name VARCHAR(24),
                            PRIMARY KEY(genre_id)
                        )
                        """
    
    TABLES['genre_instances'] = """
                                CREATE TABLE genre_instances (
                                    genre_instance_id INTEGER AUTO_INCREMENT,
                                    movie_id VARCHAR(24),
                                    genre_id VARCHAR(24),
                                    PRIMARY KEY (genre_instance_id)
                                )
                                """

    cnx, cursor = connect_to_db()
    
    
    # create the table using
    for table_name in TABLES:
        create_table_q = TABLES[table_name]
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(create_table_q)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")

    cursor.close()
    cnx.close()
    

create_tables()

Creating table movies: already exists.
Creating table genres: OK
Creating table genre_instances: already exists.


In [31]:
#inserting genre instances based on the genre_ids in a movie_data and relating them to genre_data
def insert_genre_instances(movie_data):
    cnx, cursor = connect_to_db()
    #first iterate through each movie data
    for movie in movie_data:
    #then iterate through each 'genres' list
        genres = movie['genres']
        movie_id = movie['movie_id']
        #create row where movie_id = movie_id and genre_id = genre_id
        for genre in genres:
            insert_gi_q = f"""INSERT INTO genre_instances (movie_id, genre_id)
                             VALUES('{movie_id}', '{genre}')"""
            cursor.execute(insert_gi_q)
    
    cursor.close()
    cnx.close()

In [33]:
def get_genres():
    """This function retrieves a list of a dictionary containing a list of all genres and their ids."""
    
    url = f'https://api.themoviedb.org/3/genre/movie/list?api_key={config.api_key}'
    headers = {'api_key': config.api_key}

    params = {'language': ''}
    response = requests.get(url, params=params)
    genres = response.json()['genres']
    #returns list of dictionaries, each containing key and value
    return genres

all_genres = get_genres()

#takes a list of dicts as genres as data
def insert_genres(genre_data):
    cnx, cursor = connect_to_db()
    #go through list of genres and insert genre_id and name into DB
    for genre in genre_data:
        name = genre['name']
        genre_id = genre['id']
        insert_g_query = f"""INSERT INTO genres
                            (genre_id, name)
                            VALUES ('{genre_id}', '{name}')"""
        cursor.execute(insert_g_query)
        cnx.commit()
    cnx.close()
    cursor.close()
        

    
insert_genres(all_genres)

            


IntegrityError: 1062 (23000): Duplicate entry '28' for key 'PRIMARY'

In [35]:
def get_genres_from_db():
    cnx, cursor = connect_to_db()
    query = """SELECT * 
                FROM genres"""
    cursor.execute(query)
    return cursor.fetchall()

get_genres_from_db()

[('10402', 'Music'),
 ('10749', 'Romance'),
 ('10751', 'Family'),
 ('10752', 'War'),
 ('10770', 'TV Movie'),
 ('12', 'Adventure'),
 ('14', 'Fantasy'),
 ('16', 'Animation'),
 ('18', 'Drama'),
 ('27', 'Horror'),
 ('28', 'Action'),
 ('35', 'Comedy'),
 ('36', 'History'),
 ('37', 'Western'),
 ('53', 'Thriller'),
 ('80', 'Crime'),
 ('878', 'Science Fiction'),
 ('9648', 'Mystery'),
 ('99', 'Documentary')]